In [ ]:
"""
-For type hints
-Any means a variable or return type can be any data type (string, int, tensor, etc.).
"""
from typing import Any

import os

!pip install -U mlflow
import mlflow
from mlflow.models import infer_signature


"""
-Defines step output types for Lightning training
-In PyTorch Lightning, some functions (like training_step, validation_step, test_step) return outputs that can be different types
-Lightning gives you a ready-made type alias called STEP_OUTPUT.
           -STEP_OUTPUT = shorthand type hint for “whatever the training/validation/test step is allowed to return in Lightning.”
           -PyTorch Lightning makes writing and training deep learning models easier and cleaner.
           -It handles things like training loops, validation, logging, GPU/TPU support, etc., so you don’t have to write them manually.
"""
!pip install pytorch-lightning
from pytorch_lightning.utilities.types import STEP_OUTPUT


"""
-High-level training framework to simplify PyTorch code
"""
import pytorch_lightning as pl

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping


"""
-Core PyTorch library for tensors and computations
"""
import torch


"""
-Build neural network layers and models
"""
import torch.nn as nn


"""
-Handle custom datasets and batch loading
"""
from torch.utils.data import Dataset, DataLoader


"""
-Load and preprocess dataset (CSV, Excel, etc.)
"""
import pandas as pd


"""
-BLEUScore measures how good a machine translation is by comparing it to reference translations.
-BLEU (Bilingual Evaluation Understudy): a metric that checks how closely your model’s translations match the correct/reference translations.
-torchmetrics.text.BLEUScore: makes it easy to calculate BLEU in PyTorch projects.
-BLEU tells how accurate your translations are.”
"""
from torchmetrics.text import BLEUScore


"""
-AutoTokenizer: Converts text into numbers (tokens) the model can understand.
-AutoModelForSeq2SeqLM: Pretrained sequence-to-sequence model for tasks like translation or summarization.
-Tokenizer prepares text, Model translates or generates text.
"""
%pip install -q transformers datasets sentencepiece
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM


!pip install -U mlflow pyngrok pytorch-lightning gradio sacremoses


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.9/705.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/

In [ ]:
"""
-torch.cuda.is_available() → checks if your computer has a GPU ready.
-"cuda" → runs computations on GPU (faster).
-"cpu" → runs on the processor if no GPU is available.
-torch.device(...) → tells PyTorch where to run your model and tensors.
"""
# Chooses GPU (cuda) if available, otherwise CPU, for running your PyTorch model.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
mt_pretrained_model_name = "csebuetnlp/banglat5_nmt_en_bn"

In [ ]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained(mt_pretrained_model_name)


"""
✅ AutoModelForSeq2SeqLM is:Automatic Model for Sequence-to-Sequence Language Modeling

Let’s break it down:

AutoModel → “Auto” means it automatically picks the right pretrained model architecture (you just give the model name,
             e.g., "t5-small", "facebook/mbart-large-50", etc.).
Seq2Seq → stands for Sequence-to-Sequence (input sequence → output sequence).
          Used in tasks like translation, summarization, and text generation.
"""

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


'\n✅ AutoModelForSeq2SeqLM is:Automatic Model for Sequence-to-Sequence Language Modeling\n\nLet’s break it down:\n\nAutoModel → “Auto” means it automatically picks the right pretrained model architecture (you just give the model name,\n             e.g., "t5-small", "facebook/mbart-large-50", etc.).\nSeq2Seq → stands for Sequence-to-Sequence (input sequence → output sequence).\n          Used in tasks like translation, summarization, and text generation.\n'

In [ ]:
"""A custom dataset class for Machine Translation (MT)."""
#MTDataset → Loads data from a CSV file (your custom dataset).
MAX_LENGTH = 128
class MTDataset(Dataset):
    def __init__(self, csv_file): # __init__: called when we create the dataset object.
        self.data=pd.read_csv(csv_file) #loads the CSV file
    def __len__(self): # total number of samples in the dataset.
        return len(self.data)
    def __getitem__(self,idx): # __getitem__: fetches one sample (source + target) by index.
        src_text = str(self.data.iloc[idx]['en']) # Source text (English) from column 'en'
        tgt_text = str(self.data.iloc[idx]['bn']) # Target text (Bangla) from column 'bn'
        src_encoding=tokenizer(
            src_text, # Input sentence (English)
            max_length=MAX_LENGTH,  # integer,Maximum length of tokens (fixed size input)
            padding='max_length',
            truncation=True, # Cuts off text longer than max_length
            return_tensors='pt'# Returns PyTorch tensors instead of plain lists
         )

        tgt_encoding = tokenizer(
        tgt_text,              # The target sentence (Bangla text) that we want the model to generate.
        max_length=MAX_LENGTH ,        # Fixes the size of the sequence (like saying "all sentences must be 128 tokens long").
        padding='max_length',  # If the sentence is shorter, add [PAD] tokens until it’s 128 tokens.
        truncation=True, # Cut off extra words if the sentence is longer than 128 tokens.
        return_tensors='pt'    # Converts everything into PyTorch tensors, ready for training.
    )
        return {
    'src_input_ids': src_encoding['input_ids'].squeeze(0),        # Token IDs for source (English) sentence
    'src_attention_mask': src_encoding['attention_mask'].squeeze(0),  # Mask for source (1 = real token, 0 = padding)
    'tgt_input_ids': tgt_encoding['input_ids'].squeeze(0),        # Token IDs for target (Bangla) sentence
    'tgt_attention_mask': tgt_encoding['attention_mask'].squeeze(0)   # Mask for target
}

"""
example: How are you, dude?
input_ids: 125, 14, 145, 78
max_length = 7
input_ids: [125, 14, 145, 147, 0, 0, 0]
attention_mask: [1, 1, 1, 1, 0, 0, 0],src_attention_mask → Mask to ignore [PAD] tokens.

"""


'\nexample: How are you, dude?\ninput_ids: 125, 14, 145, 78\nmax_length = 7\ninput_ids: [125, 14, 145, 147, 0, 0, 0]\nattention_mask: [1, 1, 1, 1, 0, 0, 0],src_attention_mask → Mask to ignore [PAD] tokens.\n\n'

In [ ]:
# DataModule definition
class MTDataModule(pl.LightningDataModule):
    def __init__(self, train_csv, val_csv, test_csv, batch_size=32):
        super().__init__() # Call parent LightningDataModule __init__
        # Save CSV file paths
        self.train_csv = train_csv
        self.val_csv = val_csv
        self.test_csv = test_csv
        # Save batch size (how many samples per batch)
        self.batch_size = batch_size

      # This method prepares datasets for train, val, test
    def setup(self, stage=None):
        # Create dataset objects using the CSV paths
        self.train_dataset = MTDataset(self.train_csv)
        self.val_dataset = MTDataset(self.val_csv)
        self.test_dataset = MTDataset(self.test_csv)

    # DataLoader for training (shuffle=True so model sees random data order every epoch)
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True
        )

    # DataLoader for validation (shuffle=False so order is fixed)
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False
        )

    # DataLoader for testing (also no shuffle)
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False
        )

#  Create DataModule object
data_module = MTDataModule(
    train_csv='train.csv',   # path to training data CSV
    val_csv='val.csv',       # path to validation data CSV
    test_csv='test.csv',     # path to testing data CSV
    batch_size=32            # how many samples per batch
)

In [ ]:
# Machine Translation Model
class MTModel(pl.LightningModule):
    def __init__(self, learning_rate=1e-5):
        super().__init__()

        # Load a pretrained Seq2Seq model
        self.model = AutoModelForSeq2SeqLM.from_pretrained(mt_pretrained_model_name)

        # Load tokenizer for the same model (T5-specific)
        self.tokenizer = T5Tokenizer.from_pretrained(mt_pretrained_model_name)

        #Define learning rate (small value because we are fine-tuning a pretrained model)
        self.learning_rate = learning_rate

        # Define loss function (CrossEntropyLoss)
        # "ignore_index=pad_token_id" means padding tokens won't be counted in loss.
        self.loss_fn = nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)

        # Define evaluation metric (BLEU Score)
        # BLEU checks how close translations are to the target sentences.
        self.bleu = BLEUScore()

    # Forward pass: how the model processes one batch of data
    def forward(self,
                src_input_ids,        # English tokens
                src_attention_mask,   # Mask for English (ignore PAD tokens)
                tgt_input_ids,        # Bangla tokens
                tgt_attention_mask    # Mask for Bangla
        ):
        #Call the underlying HuggingFace seq2seq model
        outputs = self.model(
            input_ids=src_input_ids,                # Source sentence (English)
            attention_mask=src_attention_mask,      # Mask for English
            decoder_input_ids=tgt_input_ids[:, :-1],# Decoder input (Bangla shifted right, teacher forcing)
            decoder_attention_mask=tgt_attention_mask[:, :-1] # Mask for Bangla
        )
        return outputs   # Contains logits (predicted token probabilities)

 # Training loop: runs for every batch during training
    def training_step(self, batch, batch_idx):
        loss = self.compute_loss(batch, batch_idx, 'train')   # Compute loss
        self.log('train_loss', loss, prog_bar=True)           # Log train loss on progress bar
        return loss

# Validation loop: runs after each epoch on validation data
    def validation_step(self, batch, batch_idx):
        loss = self.compute_loss(batch, batch_idx, 'val')     # Compute validation loss
        self.log('val_loss', loss, prog_bar=True)             # Log validation loss
        return loss

# Test loop: runs on test data
    def test_step(self, batch, batch_idx):
        loss = self.compute_loss(batch, batch_idx, 'test')    # Compute test loss
        self.log('test_loss', loss, prog_bar=True)            # Log test loss
        return loss

 # Optimizer + Scheduler setup
    def configure_optimizers(self):

        # Use AdamW optimizer (works well with transformers)
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)

        # Use learning rate scheduler (Cosine Annealing: decreases LR smoothly)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=10   # Number of epochs to restart cycle
        )

        return {'optimizer': optimizer, 'lr_scheduler': scheduler}

# ⚡ Compute loss + BLEU (shared by train/val/test)
    def compute_loss(self, batch, batch_idx, stage):
        # Unpack batch
        src_input_ids = batch['src_input_ids']
        src_attention_mask = batch['src_attention_mask']
        tgt_input_ids = batch['tgt_input_ids']
        tgt_attention_mask = batch['tgt_attention_mask']

        # Forward pass through model
        outputs = self(
            src_input_ids,
            src_attention_mask,
            tgt_input_ids,
            tgt_attention_mask
        )

        # Get predicted token logits (probabilities before softmax)
        logits = outputs.logits

        #Compute CrossEntropy loss
        # Shift target tokens by one position (teacher forcing)
        loss = self.loss_fn(
            logits.view(-1, logits.size(-1)), # Predictions: flatten for all tokens
            tgt_input_ids[:, 1:].contiguous().view(-1) # Targets: shifted right by 1
        )

        #If validation/test → also compute BLEU score
        if stage == 'val' or stage == 'test':
            preds = torch.argmax(logits, dim=-1)   # Pick highest probability tokens
            pred_texts = self.tokenizer.batch_decode(preds, skip_special_tokens=True)   # Convert to text
            tgt_texts = self.tokenizer.batch_decode(tgt_input_ids[:, 1:], skip_special_tokens=True)

            # Compute BLEU score (higher = better translation quality)
            bleu_score = self.bleu(pred_texts, [[tgt] for tgt in tgt_texts])

            # Log BLEU score to progress bar
            self.log(f'{stage}_bleu', bleu_score, prog_bar=True)

        return loss



In [ ]:
# Initialize Machine Translation model
model = MTModel()

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import os
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=2,
    verbose=True
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    save_top_k=1,
    mode='max'
)

checkpoint_path = os.path.join(
    os.getcwd(), "checkpoints", "best_model.pth"
)
if not os.path.exists(os.path.dirname(checkpoint_path)):
    os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)


In [ ]:

trainer = pl.Trainer(
    max_epochs = 2,
    accelerator = 'gpu' if torch.cuda.is_available() else 'cpu',
    devices = 1,
    precision = 32,
    log_every_n_steps = 10,
    val_check_interval = 0.25,
    callbacks = [checkpoint_callback, early_stopping]
)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
import torch

torch.save(model.state_dict(), 'mt_model_weights.pt')

In [ ]:
from google.colab import files
files.download('mt_model_weights.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import mlflow
#MLflow Tracking

EPOCHS = 1
BATCH_SIZE = 16
LEARNING_RATE = 1e-5

mlflow.set_experiment("English-Bangla-Translation")




2025/09/19 04:29:46 INFO mlflow.tracking.fluent: Experiment with name 'English-Bangla-Translation' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/467701457619767597', creation_time=1758256186583, experiment_id='467701457619767597', last_update_time=1758256186583, lifecycle_stage='active', name='English-Bangla-Translation', tags={}>

In [ ]:
data_module = MTDataModule("train.csv", "val.csv", "test.csv", batch_size=BATCH_SIZE)
model = MTModel(learning_rate=LEARNING_RATE)

In [ ]:
import mlflow
import mlflow.pytorch
from mlflow.models import infer_signature
from pyngrok import ngrok
import torch
import numpy as np

# ---------------- TRAIN + LOG ---------------- #
with mlflow.start_run() as run:
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.log_param("learning_rate", LEARNING_RATE)
    mlflow.log_param("epochs", EPOCHS)

    # Train & test
    trainer.fit(model=model, datamodule=data_module)
    evaluation_score = trainer.test(model=model, dataloaders=data_module.test_dataloader())
    mlflow.log_metric("test_loss", evaluation_score[0]['test_loss'])

    # Make sample input & output
    sample_batch = next(iter(data_module.test_dataloader()))
    sample_input = {
        'src_input_ids': sample_batch['src_input_ids'],
        'src_attention_mask': sample_batch['src_attention_mask']
    }

    with torch.no_grad():
        model.eval()
        sample_output = model(
            sample_input['src_input_ids'].to(model.device),
            sample_input['src_attention_mask'].to(model.device),
            sample_batch['tgt_input_ids'].to(model.device),
            sample_batch['tgt_attention_mask'].to(model.device)
        ).logits
        model.train()

    sample_input_np = {k: v.cpu().numpy().tolist() for k, v in sample_input.items()}
    sample_output_np = sample_output.cpu().numpy().tolist()

    # Save model to MLflow
    signature = infer_signature(sample_input_np, sample_output_np)
    mlflow.pytorch.log_model(
        pytorch_model=model,
        artifact_path="mt_model",
        signature=signature,
        input_example=sample_input_np
    )

    RUN_ID = run.info.run_id
    print("✅ Your MLflow run ID:", RUN_ID)



FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
   # ---------------- START MLFLOW UI ---------------- #
!mlflow ui --port 5000 &>/dev/null &



In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(5000)
print("🔗 MLflow UI URL:", public_url.public_url)

In [ ]:
!pip install gradio


In [ ]:
import torch
import gradio as gr
import mlflow.pytorch
from transformers import T5Tokenizer

# ---------------- CONFIG ---------------- #
mt_pretrained_model_name = "csebuetnlp/banglat5_nmt_en_bn"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LENGTH = 128

# ---------------- LOAD TOKENIZER ---------------- #
tokenizer = T5Tokenizer.from_pretrained(mt_pretrained_model_name)

# ---------------- LOAD MODEL FROM MLFLOW ---------------- #
RUN_ID = "f5ec797e21654b46a383cfc582813440"
logged_model_uri = f"runs:/{RUN_ID}/mt_model"
print("Loading model from MLflow:", logged_model_uri)

model = mlflow.pytorch.load_model(logged_model_uri)
model.to(device)
model.eval()

# ---------------- TRANSLATION FUNCTION ---------------- #
def translate_english_to_bangla(sentence: str) -> str:
    input_ids = tokenizer(
        sentence,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH
    ).input_ids.to(device)

    with torch.no_grad():
        output_tokens = model.model.generate(
            input_ids,
            max_length=MAX_LENGTH,
            num_beams=4,
            early_stopping=True
        )

    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

# ---------------- GRADIO INTERFACE ---------------- #
gr_interface = gr.Interface(
    fn=translate_english_to_bangla,
    inputs=gr.Textbox(lines=3, placeholder="Enter English sentence here...", label="English Text"),
    outputs=gr.Textbox(label="Bangla Translation"),
    title="English → Bangla Translator (from MLflow)",
    description="Translates English into Bangla using model loaded directly from MLflow."
)

# ---------------- LAUNCH ---------------- #
gr_interface.launch(share=True)
